# Chapter 3. Administration and Configuration

## 3.1 How Redis works

(1) Redis client sends commands to Redis server using Redis protocol over TCP.

(2) Redis server interprets and executes the incoming commands.

(3) Redis server stores the data in the memory by default and occasionally write the data persistently into the disk.

## 3.2 Linux setup options

(1) Real hardware

(2) Virtual machine

(3) Cloud virtual private server

* Digital Ocean
* Linode

## 3.3 A Linux setup

(1) Use a Linux VPS from Digital Ocean.

(2) SSH into the Linux VPS.

If you are on Windows, use Putty.

### 3.3.1 About Digital Ocean

Monthly plan

### 3.3.2 Creating a droplet.

### 3.3.3 Connection to the VPS

Use Putty on Windows.

### 3.3.4 Getting the tools.

Install the build tools (i.e., build-essential) for building Redis.

### 3.3.5 Build Redis.

Download the Redis source files and then build Redis.

### 3.3.6 Testing Redis.

(1) Run `redis-server` on the cloud server.

(2) Run `redis-cli` on your local machine with the Redis server IP address.

```bash
$ redis-cli -h <redis-server-ip-address>
```

To shut down the connection at the Redis client,

```redis
127.0.0.1:6379> shutdown
not connected> 
```

### 3.3.7 Better install.

Run redis-server as a daemon or a service.

https://redis.io/topics/quickstart

## 3.4 Configuration

How to configure Redis:

* redis.conf file:

Save permanent settings

* Command line:

Pass configuration objects when you start redis-server.

* Redis commands:

config get/config set when Redis is running, but changes are not saved when Redis is restarted.

### 3.4.1 Running with a config file.

`/etc/redis/redis.conf`

```bash
$ redis-server redis.conf
```

### 3.4.2 Persistence

Two modes of persistence

* RDB snapshots

Default mode; complete backup of data in the memory; snapshots are taken when a certain number of keys are changed over a certain number of time; fast and run in another thread.

* Append-only mode

Log every command with a performance penalty.

Two modes can be enabled at the same time.

### 3.4.3 Configuring persistence.

(1) RDB snapshots

```
#   In the example below the behaviour will be to save:
#   after 900 sec (15 min) if at least 1 key changed
#   after 300 sec (5 min) if at least 10 keys changed
#   after 60 sec if at least 10000 keys changed

save 900 1
save 300 10
save 60 10000
```

(2) Append-only mode

```
appendonly no

# The name of the append only file (default: "appendonly.aof")

appendfilename "appendonly.aof"
```

### 3.4.4 Replication

(1) By default Redis slaves are read-only.

(2) Write to Redis master only.

(3) Replication configuration in Section "REPLICATION" of redis.conf:

```
# slaveof <masterip> <masterport>
# masterauth <master-password>
slave-serve-stale-data yes
slave-read-only yes
```

(4) Running replication

```bash
$ redis-server --slaveof <master-server-ip> <port-number>
```

### 3.4.5 Security

Security configuration in Section "SECURITY" of redis.conf:

```
# requirepass foobared

# rename-command CONFIG b840fc02d524045429941cc15f59e41cb7be6c52
# It is also possible to completely kill a command by renaming it into
# an empty string:
#
# rename-command CONFIG ""
```

In the Redis client, use command `auth` to authenticate with the password.

```redis
127.0.0.1:6379> auth 223770
OK
```

## 3.5 More about security

Redis is designed for use by trusted clients in a trusted environment, with no robust security features of its own. 

https://www.digitalocean.com/community/tutorials/how-to-secure-your-redis-installation-on-ubuntu-14-04

### 3.5.1 Securing the server with iptables.

(1) Configure a firewall as given in https://www.digitalocean.com/community/tutorials/how-to-implement-a-basic-firewall-template-with-iptables-on-ubuntu-14-04.

(2) If you have implemented the firewall rules by following the above link, you don't need to add an extra rule for Redis, because by default, all incoming traffic is dropped unless explicitly allowed.

To check what IP address Redis is listening on, and what port it is bound to,

```bash
$ sudo netstat -plunt | grep -i redis
tcp        0      0 127.0.0.1:6379          0.0.0.0:*               LISTEN      20886/redis-server
```

To display the current iptables ruleset,

```bash
$ sudo iptables -S
-P INPUT ACCEPT
-P FORWARD ACCEPT
-P OUTPUT ACCEPT

$ sudo iptables -L
Chain INPUT (policy ACCEPT)
target     prot opt source               destination         

Chain FORWARD (policy ACCEPT)
target     prot opt source               destination         

Chain OUTPUT (policy ACCEPT)
target     prot opt source               destination       
```

### 3.5.2 Binding to localhost.

By default, Redis server is only accessible from localhost.

Open the Redis configuration file for editing

```bash
$ sudo vi /etc/redis/redis.conf
```

Locate this line and uncomment it if it is commented out.

```
bind 127.0.0.1
```

### 3.5.3 Configuring a Redis password.

Security configuration in Section "SECURITY" of redis.conf:

```
requirepass xxxxxxxx
```

You may use sha256sum or sha1sum to generate a very long and strong password.

```
$ echo "digital-ocean" | sha256sum
960c3dac4fa81b4204779fd16ad7c954f95942876b9c4fb1a255667a9dbe389d  -
$ echo "digital-ocean" | sha1sum
10d9a99851a411cdae8c3fa09d7290df192441a9  -
```

### 3.5.4 Renaming dangerous commands.

Rename or completely disable certain commands that are considered dangerous.

### 3.5.5 Setting data directory ownership and file permission.

(1) `/var/lib/redis`

```bash
$ sudo chmod 700 /var/lib/redis
```

(2) `/etc/redis/redis.conf`

```bash
$ sudo chown redis:root /etc/redis/redis.conf
$ sudo chmod 600 /etc/redis/redis.conf
```

## 3.6 Monitor

### 3.6.1 Command `monitor`

```redis
127.0.0.1:6379> monitor
OK
1517823428.502909 [0 127.0.0.1:60948] "COMMAND"
1517823439.001239 [0 127.0.0.1:60948] "get" "test"
```

### 3.6.2 Slow log

(1) Configure the slow log.

```redis
## Configure Redis to log every command which takes longer than 2 microseconds.
127.0.0.1:6379> config set slowlog-log-slower-than 2
OK

## If the time threshold is 0, it will simply log every command.
127.0.0.1:6379> config set slowlog-log-slower-than 0
OK
```

(2) Show the slow log.

```redis
127.0.0.1:6379> slowlog get
1) 1) (integer) 1
   2) (integer) 1517823587
   3) (integer) 5
   4) 1) "get"
      2) "test"
   5) "127.0.0.1:60948"
   6) ""
2) 1) (integer) 0
   2) (integer) 1517823580
   3) (integer) 10
   4) 1) "config"
      2) "set"
      3) "slowlog-log-slower-than"
      4) "0"
   5) "127.0.0.1:40628"
   6) ""
127.0.0.1:6379> slowlog get 1
1) 1) (integer) 2
   2) (integer) 1517823638
   3) (integer) 24
   4) 1) "slowlog"
      2) "get"
   5) "127.0.0.1:40628"
   6) ""
```

## 3.7 Other Redis tools

* Redis React (previously called RedisAdminUI)

https://github.com/ServiceStackApps/RedisReact